# Análise de dados

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../datasets/application_data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [10]:
df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df.select_dtypes(include="object").head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,reg oper account,block of flats,Panel,No
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,reg oper account,block of flats,Panel,No


## Retirar variaveis que não pertence ao estudo do caso (Drop axi=1)

In [9]:
df = df.drop("SK_ID_CURR", axis=1)

## Identificar variaveis Nulas e preencher com a media da categoria (Nan)

In [38]:
df.select_dtypes(include="object").isnull().sum()

NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
NAME_TYPE_SUITE               0
NAME_INCOME_TYPE              0
NAME_EDUCATION_TYPE           0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
OCCUPATION_TYPE               0
WEEKDAY_APPR_PROCESS_START    0
ORGANIZATION_TYPE             0
FONDKAPREMONT_MODE            0
HOUSETYPE_MODE                0
WALLSMATERIAL_MODE            0
EMERGENCYSTATE_MODE           0
dtype: int64

In [26]:
df["NAME_TYPE_SUITE"].fillna(axis=0, method="bfill", inplace=True)
df["OCCUPATION_TYPE"].fillna(axis=0, method="bfill", inplace=True)
df["FONDKAPREMONT_MODE"].fillna(axis=0, method="bfill", inplace=True)
df["HOUSETYPE_MODE"].fillna(axis=0, method="bfill", inplace=True)
df["WALLSMATERIAL_MODE"].fillna(axis=0, method="bfill", inplace=True)
df["EMERGENCYSTATE_MODE"].fillna(axis=0, method="bfill", inplace=True)
df["FONDKAPREMONT_MODE"].fillna(axis=0, method="bfill", inplace=True)

In [37]:
df["FONDKAPREMONT_MODE"].fillna(value="reg oper account", inplace=True)

In [ ]:
# df.dropna(axis=1)

In [14]:
df.select_dtypes(include="int64").isnull().sum()

TARGET                         0
CNT_CHILDREN                   0
DAYS_BIRTH                     0
DAYS_EMPLOYED                  0
DAYS_ID_PUBLISH                0
FLAG_MOBIL                     0
FLAG_EMP_PHONE                 0
FLAG_WORK_PHONE                0
FLAG_CONT_MOBILE               0
FLAG_PHONE                     0
FLAG_EMAIL                     0
REGION_RATING_CLIENT           0
REGION_RATING_CLIENT_W_CITY    0
HOUR_APPR_PROCESS_START        0
REG_REGION_NOT_LIVE_REGION     0
REG_REGION_NOT_WORK_REGION     0
LIVE_REGION_NOT_WORK_REGION    0
REG_CITY_NOT_LIVE_CITY         0
REG_CITY_NOT_WORK_CITY         0
LIVE_CITY_NOT_WORK_CITY        0
FLAG_DOCUMENT_2                0
FLAG_DOCUMENT_3                0
FLAG_DOCUMENT_4                0
FLAG_DOCUMENT_5                0
FLAG_DOCUMENT_6                0
FLAG_DOCUMENT_7                0
FLAG_DOCUMENT_8                0
FLAG_DOCUMENT_9                0
FLAG_DOCUMENT_10               0
FLAG_DOCUMENT_11               0
FLAG_DOCUM

In [29]:
df.select_dtypes(include="float64").isnull().sum()

AMT_INCOME_TOTAL              0
AMT_CREDIT                    0
AMT_ANNUITY                   0
AMT_GOODS_PRICE               0
REGION_POPULATION_RELATIVE    0
                             ..
AMT_REQ_CREDIT_BUREAU_DAY     0
AMT_REQ_CREDIT_BUREAU_WEEK    0
AMT_REQ_CREDIT_BUREAU_MON     0
AMT_REQ_CREDIT_BUREAU_QRT     0
AMT_REQ_CREDIT_BUREAU_YEAR    0
Length: 65, dtype: int64

In [18]:
df["AMT_ANNUITY"].fillna(value=df["AMT_ANNUITY"].mean(), inplace=True)
df["AMT_GOODS_PRICE"].fillna(value=df["AMT_GOODS_PRICE"].mean(), inplace=True)
df["AMT_REQ_CREDIT_BUREAU_DAY"].fillna(value=df["AMT_REQ_CREDIT_BUREAU_DAY"].mean(), inplace=True)
df["AMT_REQ_CREDIT_BUREAU_WEEK"].fillna(value=df["AMT_REQ_CREDIT_BUREAU_WEEK"].mean(), inplace=True)
df["AMT_REQ_CREDIT_BUREAU_MON"].fillna(value=df["AMT_REQ_CREDIT_BUREAU_MON"].mean(), inplace=True)
df["AMT_REQ_CREDIT_BUREAU_QRT"].fillna(value=df["AMT_REQ_CREDIT_BUREAU_QRT"].mean(), inplace=True)
df["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(value=df["AMT_REQ_CREDIT_BUREAU_YEAR"].mean(), inplace=True)


In [30]:
df.fillna(value=df.mean(), inplace=True)

In [39]:
df.isnull().sum().sum()

0

## Converter variaveis textos/categoricos para numericos/categoricos 

In [40]:
df.select_dtypes(include="object").isnull().sum()

NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
NAME_TYPE_SUITE               0
NAME_INCOME_TYPE              0
NAME_EDUCATION_TYPE           0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
OCCUPATION_TYPE               0
WEEKDAY_APPR_PROCESS_START    0
ORGANIZATION_TYPE             0
FONDKAPREMONT_MODE            0
HOUSETYPE_MODE                0
WALLSMATERIAL_MODE            0
EMERGENCYSTATE_MODE           0
dtype: int64

In [44]:
def convert_text_to_numetic(value, lstText):
    for index, text in enumerate(lstText):
        if text == value:
            return index


In [51]:
lst_Coll = ["CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "OCCUPATION_TYPE", "WEEKDAY_APPR_PROCESS_START", "ORGANIZATION_TYPE", "FONDKAPREMONT_MODE", "HOUSETYPE_MODE", "WALLSMATERIAL_MODE", "EMERGENCYSTATE_MODE"] 

for col in lst_Coll:
    lstText = df[col].unique()
    df["CAT_" + col] = [convert_text_to_numetic(text, lstText) for text in df[col].values]

In [52]:
df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CAT_NAME_EDUCATION_TYPE,CAT_NAME_FAMILY_STATUS,CAT_NAME_HOUSING_TYPE,CAT_OCCUPATION_TYPE,CAT_WEEKDAY_APPR_PROCESS_START,CAT_ORGANIZATION_TYPE,CAT_FONDKAPREMONT_MODE,CAT_HOUSETYPE_MODE,CAT_WALLSMATERIAL_MODE,CAT_EMERGENCYSTATE_MODE
0,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,0,0,0,0,0
1,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,1,1,0,1,1,1,0,0,1,0
2,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,1,2,0,0,2,0
3,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,2,0,0,0,0,0,0,2,0
4,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,1,2,3,0,0,2,0


## Retirar registros que tenha variaveis com outliers (Escor Z)

In [55]:
name_columns = df.select_dtypes(include="float64").columns
des_target = df["TARGET"].unique()

In [58]:
name_columns = df.select_dtypes(include="int64").columns

In [59]:
for variavel in name_columns:
    df_defaut = []
    for value, target in enumerate(des_target):
        dados = df[df["TARGET"]==value]
        print(f"Classe: {des_target[value]} - variavel: {variavel}")

        data_mean, data_std = dados[variavel].mean(), dados[variavel].std()
        data_min, data_max = dados[variavel].min(), dados[variavel].max()
        print("Real Min: %.3f Real Max: %.3f" %  (data_min, data_max))

        cut_off = data_std * 2.5
        lower, upper = data_mean - cut_off, data_mean + cut_off
        print("Limit Min: %.3f Limit Max: %.3f" %  (lower, upper))


        dataset = dados[dados[variavel] >= lower]
        if len(dataset) == 0:
            dataset = dados[dados[variavel] <= upper]
        else:
            dataset = dataset[dataset[variavel] <= upper]


        df_defaut.append(dataset)

        outliers = pd.concat([dados[dados[variavel] < lower], dados[dados[variavel] > upper]])
        print("Identfied outliers: %d \n" % len(outliers))

    df = pd.concat(df_defaut)

Classe: 1 - variavel: TARGET
Real Min: 0.000 Real Max: 0.000
Limit Min: 0.000 Limit Max: 0.000
Identfied outliers: 0 

Classe: 0 - variavel: TARGET
Real Min: 1.000 Real Max: 1.000
Limit Min: 1.000 Limit Max: 1.000
Identfied outliers: 0 

Classe: 1 - variavel: CNT_CHILDREN
Real Min: 0.000 Real Max: 3.000
Limit Min: -1.267 Limit Max: 2.064
Identfied outliers: 89 

Classe: 0 - variavel: CNT_CHILDREN
Real Min: 0.000 Real Max: 3.000
Limit Min: -1.272 Limit Max: 2.174
Identfied outliers: 14 

Classe: 1 - variavel: DAYS_BIRTH
Real Min: -25197.000 Real Max: -7673.000
Limit Min: -26884.932 Limit Max: -5324.654
Identfied outliers: 0 

Classe: 0 - variavel: DAYS_BIRTH
Real Min: -25168.000 Real Max: -7680.000
Limit Min: -25160.024 Limit Max: -4256.516
Identfied outliers: 1 

Classe: 1 - variavel: DAYS_EMPLOYED
Real Min: -17522.000 Real Max: 365243.000
Limit Min: -294063.184 Limit Max: 430952.805
Identfied outliers: 0 

Classe: 0 - variavel: DAYS_EMPLOYED
Real Min: -14540.000 Real Max: 365243.000
L

In [60]:
import collections

In [61]:
print(collections.Counter(df.TARGET))

Counter({0: 32088, 1: 3942})


## Balancear as categorias (SMOTE)

In [69]:
name = df.select_dtypes(include="object").columns

In [70]:
name

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [76]:
df = df.drop(name, axis=1)


In [88]:
y = df["TARGET"].to_numpy()
X = df
X = X.drop("TARGET", axis=1)
X = X.to_numpy()

In [79]:
from imblearn.over_sampling import SMOTE

In [90]:
overSampling = SMOTE()
X, y = overSampling.fit_resample(X, y)
print(collections.Counter(y))

Counter({0: 32088, 1: 32088})


In [94]:
name_coll = df.columns[1:]
df_smote = pd.DataFrame(X, columns=name_coll)

In [95]:
df_smote.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CAT_NAME_EDUCATION_TYPE,CAT_NAME_FAMILY_STATUS,CAT_NAME_HOUSING_TYPE,CAT_OCCUPATION_TYPE,CAT_WEEKDAY_APPR_PROCESS_START,CAT_ORGANIZATION_TYPE,CAT_FONDKAPREMONT_MODE,CAT_HOUSETYPE_MODE,CAT_WALLSMATERIAL_MODE,CAT_EMERGENCYSTATE_MODE
0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,-2437.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,0.0,99000.0,490495.5,27517.5,454500.0,0.035792,-16941.0,-1588.0,-4970.0,-477.0,...,0.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0
2,1.0,112500.0,652500.0,21177.0,652500.0,0.022800,-10197.0,-679.0,-4427.0,-738.0,...,1.0,1.0,0.0,1.0,4.0,7.0,0.0,0.0,2.0,0.0
3,1.0,90000.0,544491.0,17563.5,454500.0,0.015221,-11348.0,-2038.0,-1021.0,-3964.0,...,1.0,0.0,0.0,1.0,1.0,13.0,0.0,0.0,0.0,0.0
4,1.0,202500.0,1132573.5,37561.5,927000.0,0.025164,-14815.0,-1652.0,-2299.0,-2299.0,...,0.0,1.0,0.0,5.0,1.0,14.0,0.0,0.0,2.0,0.0


## Normalizar as variaveis (StandardScaler)

In [96]:
from sklearn import preprocessing

In [97]:
scaler = preprocessing.StandardScaler().fit(X)

In [98]:
scaler

StandardScaler()

In [99]:
X_scaled = scaler.transform(X)

In [101]:
df_scaled = pd.DataFrame(X_scaled, columns=name_coll)

In [102]:
df_scaled.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,CAT_NAME_EDUCATION_TYPE,CAT_NAME_FAMILY_STATUS,CAT_NAME_HOUSING_TYPE,CAT_OCCUPATION_TYPE,CAT_WEEKDAY_APPR_PROCESS_START,CAT_ORGANIZATION_TYPE,CAT_FONDKAPREMONT_MODE,CAT_HOUSETYPE_MODE,CAT_WALLSMATERIAL_MODE,CAT_EMERGENCYSTATE_MODE
0,-0.714514,-0.093496,-0.750931,0.457481,-0.641849,-1.082484,-1.040202,-0.306875,-1.833544,0.276187,...,-0.423352,1.618658,-0.298383,-1.115990,-1.471774,-1.065326,-0.506604,0.0,0.867925,0.0
1,-0.714514,-0.666964,-0.144293,0.241044,-0.038855,2.025654,-0.483101,-0.292469,-0.196959,1.668294,...,-0.423352,-0.086901,-0.298383,-1.115990,-1.471774,-0.593965,-0.506604,0.0,0.867925,0.0
2,0.845682,-0.451913,0.408411,-0.391651,0.719194,0.571690,1.337197,-0.283444,-0.014219,1.482917,...,2.680314,-0.086901,-0.298383,-0.812719,0.511533,-0.240444,-0.506604,0.0,0.867925,0.0
3,0.845682,-0.810331,0.039921,-0.752229,-0.038855,-0.276492,1.026526,-0.296937,1.132030,-0.808378,...,2.680314,-1.792459,-0.298383,-0.812719,-0.975947,0.466597,-0.506604,0.0,-1.124547,0.0
4,0.845682,0.981757,2.046258,1.243299,1.770126,0.836251,0.090736,-0.293104,0.701934,0.374203,...,-0.423352,-0.086901,-0.298383,0.400363,-0.975947,0.584437,-0.506604,0.0,0.867925,0.0


## Identificar as melhores relações de variaveis para a variavel taget (random forest)

In [103]:
from sklearn.ensemble import RandomForestClassifier

In [104]:
clf = RandomForestClassifier(random_state=15, n_estimators=1000)
clf.fit(df_scaled, y)

RandomForestClassifier(n_estimators=1000, random_state=15)

In [110]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 15,
 'verbose': 0,
 'warm_start': False}

## Aplicar o PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(X)
